<a href="https://colab.research.google.com/github/dadakhalandar/Lab10/blob/main/IE630_Team_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TOPIC:2 
Q)Make ten indepedent replications for the (M,L)

In [78]:
import numpy as np
import random as r

In [79]:
def quantity(I,M,L,lead_time):
  if I < L and 0<= lead_time <= 1:
    Q = M-I
  elif lead_time ==20 and I < L:
    Q = M-I
  else:
    Q = 0
  return Q 

In [80]:
def ord_cost(Q):
  if Q > 0:
   return 60 + 5*Q
  else:
    return 0

In [81]:
def back_cost(I,D):
  if I-D < 0:
    return (D-I)*4
  else:
    return 0 

In [82]:
def inv_cost(I):
  if I > 0:
    return I
  else:
    return 0

In [83]:
def sell(inv,monthly_demand,order_reciept):
  b = [50,*inv]
  revenue = []
  for i in range(2):
    if order_reciept[i]+b[i] > 0:
      sales = min(order_reciept[i]+b[i],monthly_demand[i])*10
      revenue.append(sales)

  for i in range(2,len(inv),1):
    z = b[i-2] - monthly_demand[i-2] - monthly_demand[i-1] 
    if z > 0 :
      sales = min(z,monthly_demand[i])*5
      if monthly_demand[i] > z:
        sales = sales+ 10*min(order_reciept[i]+b[i]-z,monthly_demand[i]-z)
    elif order_reciept[i]+b[i] > 0:
      sales = min(order_reciept[i]+b[i],monthly_demand[i])*10
    else:
      sales = 0
    revenue.append(sales)
  return revenue

In [84]:
def simulation(no_of_replication = 1, length_of_each_run = 100 ):
  L = length_of_each_run
  n = no_of_replication
  mean_profit = []
  mean_monthly_cost = []
  for i in range(n):
    z = -1
    #r.seed(a=10*i)
    order_reciept = int(L+2)*[0] 
    I = 50
    monthly_cost = []
    monthly_orders = []
    monthly_demand = []
    t = 0 #initial month
    L_time = 20
    inv = []
    lead = []
    while t < L: # run for 100 month
      I = I + order_reciept[t] 
      d_t = 0 
      dm = [] 
      while d_t < 1: 
        d_t = d_t + r.expovariate(15)
        pr = r.uniform(0,1)
        if pr >= 0 and pr<= 0.5:
          D = 1
        elif 0.5 < pr <= 0.75:
          D = 2
        elif 0.75 < pr <= 7/8:  
          D = 3
        else:
          D = 4
        dm.append(D)
      demand = sum(dm)

      I = I -demand
      if t > 4:
        z = inv[t-4]  -  monthly_demand[t-3] -  monthly_demand[t-2] 
      if z > 0:
        I = I - z
      Q = quantity(I,50,30,L_time)
      monthly_cost.append(ord_cost(Q)+back_cost(I,demand)+ inv_cost(I))
      inv.append(I)
      monthly_orders.append(Q)
      monthly_demand.append(demand)
      if Q > 0:
        L_time = r.uniform(0.25,1.25)
      else:
        L_time = 0
        lead.append(L_time)
      if 0.25 <= L_time <= 1 and Q > 0:
        order_reciept[t+1] = Q
      elif L_time > 1 and Q > 0: 
        order_reciept[t+2] = Q
        order_reciept[t+1] = 0
      t = t + 1 
    seling = sell(inv,monthly_demand,order_reciept)
    mean_profit.append(-np.mean(monthly_cost)+ np.mean(seling)) 
    mean_monthly_cost.append(np.mean(monthly_cost)) 
  return mean_profit,mean_monthly_cost

In [85]:
mean_profit,mean_monthly_cost = simulation(10)

In [86]:
mean_profit

[-21.129999999999995,
 -51.95000000000002,
 -37.92999999999998,
 -70.72999999999996,
 -61.870000000000005,
 -87.37,
 -81.84,
 -84.39999999999998,
 -60.94,
 -73.62]

In [87]:
mean_monthly_cost

[297.13, 304.85, 281.33, 316.03, 315.92, 308.32, 305.44, 314.9, 304.14, 312.92]

In [88]:
import statistics

In [89]:
#90% CI FOR MEAN COST
S = (statistics.variance(mean_monthly_cost))**0.5
R = 10
t = 1.833 #alpha = 0.1, 
hw_cost  = t*S/np.sqrt(R)
print(hw_cost)

6.182195585314007


In [90]:
mean_mc = np.mean(mean_monthly_cost)
print(mean_mc)

306.098


In [91]:
#90% CI FOR MEAN profit
S = (statistics.variance(mean_profit))**0.5
R = 10
t = 1.833 #alpha = 0.1, 
hw_profit  = t*S/np.sqrt(R)
print(hw_profit)

12.354063064477854


In [92]:
mean_mp = np.mean(mean_profit)
print(mean_mp)

-63.178
